In [1]:
import pandas as pd 
import numpy as np 
import xlrd 
import sklearn

In [16]:
# extract three groups of flip features from the paper

# P1(mHSPC) testtosterone suppression failure: CER(ceramide), DG(diacylglycerol), TG(triaclyglycerol)
# P2(mCRPC) overall survival:  CER(ceramide), Sph(Sphingosine), Acylcarnitine(acylcarnitine)

# prognastic 3-lipid signature: P = sigmoid(y) 
# y =  3.132 * Cer(d18:1/24:1) + 2.1724 * Sph(d18:2) + 1.8593* PI(16:0/16:0) - 91.217

In [2]:
table_specie = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',0)
table_total= pd.read_excel('../data/raw/LCMS_data_copy.xlsx',1)
lip_name_table = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',4)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/LCMS_data_copy.xlsx'

In [18]:
# it seems prognastic 3-lipid signature will be always > 0. 
# some unknown data-process used in paper


print(table_specie['Cer(d18:1/24:1)'].median())
print(table_specie['Sph(d18:2)'].median())
print(table_specie['PI(16:0/16:0)'].median())

1753.1276496932069
16.5595771642514
141.84040836080527


In [19]:
lip_name_table

,Original name,Updated name,Class,BioGrouping,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Abreviation,old abbreviation,Lipid class
0,Sph(d16:1) Results,Sph(d16:1),Sph,1,NaN,NaN,NaN,NaN,FFA,FFA,free fatty acids
1,Sph(d18:1) Results,Sph(d18:1),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:0/xx:xx),dhCer,dihydroceramide
2,Sph(d18:2) Results,Sph(d18:2),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:1/xx:x),Cer,ceramide
3,S1P(d16:1) Results,S1P(d16:1),S1P,2,NaN,NaN,NaN,NaN,Hex1Cer,MHC,monohexosylceramide
4,S1P(d18:0) Results,S1P(d18:0),S1P,2,NaN,NaN,NaN,NaN,Hex2Cer,DHC,dihexosylceramide
...,...,...,...,...,...,...,...,...,...,...,...
795,LPC 20:4_OH Results,LPC(20:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,LPC 22:6_OH Results,LPC(22:6) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,PC 34:2_OH Results,PC(34:2) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,PC 36:4(a\b)_OH Results,PC(36:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
feature_list_total = (table_total.iloc[:,2:].columns.values.tolist())[1:] # id and features name
feature_list_specie = (table_specie.iloc[:,2:].columns.values.tolist())[1:] # id and features name 
name_group_link = lip_name_table[['Updated name','Class','BioGrouping']]

In [6]:
name_group_link

,Updated name,Class,BioGrouping
0,Sph(d16:1),Sph,1
1,Sph(d18:1),Sph,1
2,Sph(d18:2),Sph,1
3,S1P(d16:1),S1P,2
4,S1P(d18:0),S1P,2
...,...,...,...
795,LPC(20:4) [+OH],OxSpecies,48
796,LPC(22:6) [+OH],OxSpecies,48
797,PC(34:2) [+OH],OxSpecies,48
798,PC(36:4) [+OH],OxSpecies,48


In [7]:
feature_list_total

['Sph',
 'S1P',
 'dhCer',
 'Cer',
 'C1P',
 'MHC',
 'DHC',
 'THC',
 'GM3',
 'GM1',
 'Sulfatide',
 'SM',
 'PA',
 'PC',
 'PC(O)',
 'PC(P)',
 'LPC',
 'LPC(O)',
 'LPC(P)',
 'PE',
 'PE(O)',
 'PE(P)',
 'LPE',
 'LPE(P)',
 'PI',
 'PIP1',
 'LPI',
 'PS',
 'PG',
 'CE',
 'COH',
 'DE',
 'FFA',
 'Acylcarnitine',
 'DG',
 'TG(SIM)',
 'TG [NL]',
 'TG(O)',
 'TG(O) [NL]',
 'Ubiquinone',
 'OxSpecies']

In [21]:
group_name_list = [name_group_link.loc[name_group_link['Updated name']==item]['Class'].values[0] for item in feature_list_specie]# get group name for each specie

In [22]:
len(group_name_list)

774

In [23]:
len(set(group_name_list)) # nunmber of total groups

41

In [24]:
group_name_dict = {}
for i,item in enumerate(set(group_name_list)):
    group_name_dict[item] = i
group_name_dict

{'PG': 0,
 'LPC(O)': 1,
 'LPE': 2,
 'PS': 3,
 'PE(O)': 4,
 'SM': 5,
 'Acylcarnitine': 6,
 'Cer': 7,
 'PA': 8,
 'DE': 9,
 'FFA': 10,
 'dhCer': 11,
 'PIP1': 12,
 'S1P': 13,
 'PC': 14,
 'DHC': 15,
 'COH': 16,
 'THC': 17,
 'TG(O)': 18,
 'LPC': 19,
 'CE': 20,
 'MHC': 21,
 'C1P': 22,
 'Sulfatide': 23,
 'TG(O) [NL]': 24,
 'Ubiquinone': 25,
 'DG': 26,
 'GM3': 27,
 'LPI': 28,
 'LPC(P)': 29,
 'TG [NL]': 30,
 'PC(P)': 31,
 'PI': 32,
 'GM1': 33,
 'LPE(P)': 34,
 'TG(SIM)': 35,
 'OxSpecies': 36,
 'PE': 37,
 'Sph': 38,
 'PC(O)': 39,
 'PE(P)': 40}

In [25]:
# P1(mHSPC) testtosterone suppression failure: CER(ceramide), DG(diacylglycerol), TG(triaclyglycerol): 'TG(SIM)','TG [NL]','TG(O)','TG(O) [NL]'
# P2(mCRPC) overall survival:  CER(ceramide), Sph(Sphingosine), Acylcarnitine(acylcarnitine)

# prognastic 3-lipid signature: P = sigmoid(y) 
# y =  3.132 * Cer(d18:1/24:1) + 2.1724 * Sph(d18:2) + 1.8593* PI(16:0/16:0) - 91.217
target_total_P1 = ['DG','Cer','TG(SIM)','TG [NL]','TG(O)','TG(O) [NL]'] 
target_total_P2 = ['Sph','Cer','Acylcarnitine'] # Cer is the duplicated one, remember to remove when do regression  


target_total_P1_counts = {item:0 for item in target_total_P1}
target_total_P2_counts = {item:0 for item in target_total_P2}

sig_feature_list = ['Cer(d18:1/24:1)','Sph(d18:2)','PI(16:0/16:0)']



In [26]:
# compute the signature features
# as the given formula of signature is not right, we remove the the bias, directly use the binary/continues signature value 
sig_feature = (3.132 * table_specie['Cer(d18:1/24:1)'] + 2.1724 * table_specie['Sph(d18:2)'] +  1.8593* table_specie['PI(16:0/16:0)'])
sig_feature = (sig_feature-sig_feature.min())/(sig_feature.max()-sig_feature.min())

In [27]:
def id_process(id):

    if id[-1] == 'C':
        # pattern 'num + C' 
        return id[:-2]
    elif id[1] == '-':
        # pattern 'P-num' 
        return id[2:]
    else:# pattern 'P num'
        return id[1:]
table_specie.iloc[:,2].apply(id_process)

0      1001
1      1010
2      1019
3      1028
4      1037
       ... 
802    2395
803    2412
804    2429
805    2456
806    2493
Name: Patient ID, Length: 807, dtype: object

In [28]:
# binarize the lip feature spicie: for each feature ,
#  if val > median, set feature_high = 1,feature_low = 0
#  else,   set feature_high = 0,feature_high = 1

table_specie_binary = pd.DataFrame()
table_specie_binary['Patient_ID'] = table_specie.iloc[:,2].apply(id_process)

for lip_name in feature_list_specie:
    df = table_specie[lip_name]
    # normalized_df = (df-df.min())/(df.max()-df.min())
    # normalized_df = (df-df.mean())/df.std()
    normalized_df = df.fillna(0)

    

    group_name = name_group_link.loc[name_group_link['Updated name']==lip_name]['Class'].values[0]

    if group_name in target_total_P1:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_binary['lip_P1_' + group_name + '_' +lip_name +'_'+'high'] =  (normalized_df>=normalized_df.median())*1
        table_specie_binary['lip_P1_' + group_name + '_' +lip_name +'_'+'low'] = (normalized_df<normalized_df.median())*1 

    if group_name in target_total_P2:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_binary['lip_P2_' + group_name + '_' +lip_name +'_'+'high'] =  (normalized_df>=normalized_df.median())*1
        table_specie_binary['lip_P2_' + group_name + '_' +lip_name +'_'+'low'] = (normalized_df<normalized_df.median())*1 

table_specie_binary['lip_signature'] = (sig_feature<sig_feature.median())*1

table_specie_binary    
table_specie_binary.to_csv('../data/processed/process_lip_specie_binary_single_P1_P2_sig.csv',index=False)

In [29]:
# the lip feature spicie: continues
# binarize the lip feature spicie: for each feature ,
#  if val > median, set feature_high = 1,feature_low = 0
#  else,   set feature_high = 0,feature_high = 1

table_specie_cont = pd.DataFrame()
table_specie_cont['Patient_ID'] = table_specie.iloc[:,2].apply(id_process)

for lip_name in feature_list_specie:
    df = table_specie[lip_name].fillna(0)

    normalized_df = (df-df.min())/(df.max()-df.min())
    # normalized_df = (df-df.mean())/df.std()
    # normalized_df = df.fillna(0)

    

    group_name = name_group_link.loc[name_group_link['Updated name']==lip_name]['Class'].values[0]

    if group_name in target_total_P1:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_cont['lip_P1_' + group_name + '_' + lip_name  ] =  normalized_df
        target_total_P1_counts[group_name] = target_total_P1_counts[group_name] + 1

    if group_name in target_total_P2:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_cont['lip_P2_' + group_name + '_' + lip_name ] =  normalized_df
        target_total_P2_counts[group_name] = target_total_P2_counts[group_name] + 1

table_specie_cont['lip_signature'] = sig_feature

table_specie_cont
table_specie_cont.to_csv('../data/processed/process_lip_specie_cont_single_P1_P2_sig.csv',index=False)

In [67]:
table_specie_cont

,Patient_ID,lip_P2_Sph_Sph(d16:1),lip_P2_Sph_Sph(d18:1),lip_P2_Sph_Sph(d18:2),lip_P1_Cer_Cer(d16:1/16:0),lip_P2_Cer_Cer(d16:1/16:0),lip_P1_Cer_Cer(d16:1/18:0),lip_P2_Cer_Cer(d16:1/18:0),lip_P1_Cer_Cer(d16:1/20:0),lip_P2_Cer_Cer(d16:1/20:0),...,lip_P1_TG(O) [NL]_TG(O-52:2) [NL-16:0],lip_P1_TG(O) [NL]_TG(O-52:2) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-52:2) [NL-18:1],lip_P1_TG(O) [NL]_TG(O-54:2) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-54:2) [NL-18:1],lip_P1_TG(O) [NL]_TG(O-54:3) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-54:3) [NL-18:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-17:1],lip_P1_TG(O) [NL]_TG(O-54:4) [NL-18:2],lip_signature
0,1001,0.495112,0.054354,0.216575,0.294450,0.294450,0.305661,0.305661,0.181769,0.181769,...,0.073178,0.097921,0.092968,0.058154,0.058486,0.064083,0.091286,0.057364,0.170173,0.291482
1,1010,0.400186,0.137942,0.276835,0.240365,0.240365,0.223917,0.223917,0.171905,0.171905,...,0.052621,0.073240,0.113669,0.059007,0.041537,0.044645,0.079611,0.020748,0.109859,0.261832
2,1019,0.368722,0.155415,0.169811,0.174938,0.174938,0.135445,0.135445,0.144821,0.144821,...,0.068379,0.087477,0.067291,0.045787,0.044786,0.063729,0.086322,0.109468,0.363641,0.186702
3,1028,0.213523,0.103455,0.163318,0.212065,0.212065,0.166905,0.166905,0.082406,0.082406,...,0.020997,0.021701,0.029004,0.012158,0.011367,0.016312,0.032278,0.010190,0.059356,0.134988
4,1037,0.241833,0.121882,0.226665,0.320932,0.320932,0.334240,0.334240,0.265990,0.265990,...,0.089657,0.123884,0.103111,0.073940,0.071691,0.067329,0.066803,0.036448,0.082580,0.308613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2395,0.350010,0.037216,0.130594,0.149554,0.149554,0.166975,0.166975,0.088372,0.088372,...,0.061499,0.038216,0.046612,0.044319,0.036906,0.032707,0.065225,0.022539,0.144121,0.189338
803,2412,0.608675,0.160250,0.327372,0.425594,0.425594,0.332021,0.332021,0.282406,0.282406,...,0.055032,0.074486,0.088612,0.060654,0.078983,0.089459,0.073796,0.103588,0.081468,0.310620
804,2429,0.341628,0.086636,0.218512,0.304081,0.304081,0.244213,0.244213,0.219390,0.219390,...,0.050136,0.050389,0.075450,0.036494,0.056994,0.075550,0.074167,0.047676,0.078162,0.209158
805,2456,0.419137,0.141534,0.113275,0.200173,0.200173,0.420504,0.420504,0.274377,0.274377,...,0.149058,0.200696,0.265532,0.118484,0.160342,0.111181,0.161699,0.148844,0.178869,0.473938


In [30]:
# binarize the lip feature spicie: for each feature with standard way,
#  if val > median, set feature = 1,else = 0

table_specie_binary = pd.DataFrame()
table_specie_binary['Patient_ID'] = table_specie.iloc[:,2].apply(id_process)

for lip_name in feature_list_specie:
    df = table_specie[lip_name]
    # normalized_df = (df-df.min())/(df.max()-df.min())
    # normalized_df = (df-df.mean())/df.std()
    normalized_df = df.fillna(0)

    

    group_name = name_group_link.loc[name_group_link['Updated name']==lip_name]['Class'].values[0]

    if group_name in target_total_P1:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_binary['lip_P1_' + group_name + '_' +lip_name ] =  (normalized_df>=normalized_df.median())*1

    if group_name in target_total_P2:
        group_name_idx = str(group_name_dict[group_name])
        table_specie_binary['lip_P2_' + group_name + '_' +lip_name ] =  (normalized_df>=normalized_df.median())*1

table_specie_binary['lip_signature'] = (sig_feature>sig_feature.median())*1

table_specie_binary    
table_specie_binary.to_csv('../data/processed/process_lip_specie_standard_binary_single_P1_P2_sig.csv',index=False)

In [65]:
target_total_P1_counts

{'DG': 25,
 'Cer': 61,
 'TG(SIM)': 34,
 'TG [NL]': 79,
 'TG(O)': 9,
 'TG(O) [NL]': 20}